In [1]:
from tf2_bert.models import build_transformer_model
from tf2_bert.tokenizers import Tokenizer
import numpy as np

In [3]:
# 定义预训练模型路径
model_dir = './chinese_roberta_wwm_ext_L-12_H-768_A-12'
# BERT 参数
config_path = model_dir+'/bert_config.json'
# 保存模型权值参数的文件
checkpoint_path = model_dir+'/bert_model.ckpt'
# 词表
dict_path = model_dir+'/vocab.txt'
# 建立分词器
tokenizer = Tokenizer(dict_path)

C:\Users\31600\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


['[CLS]', '机', '器', '学', '习', '[SEP]']


**文本特征提取**

In [ ]:
# 建立模型，加载权重
model = build_transformer_model(config_path, checkpoint_path)
# 句子 0
sentence0 = '机器学习'
# 句子 1
sentence1 = '深度学习'
# 用分词器对句子分词
tokens = tokenizer.tokenize(sentence0)
# 分词后自动在句子前加上[CLS]，在句子后加上[SEP]
print(tokens)

In [4]:
# 编码测试
token_ids, segment_ids = tokenizer.encode(sentence0)
# [CLS]的编号为 101，机为 3322，器为 1690，学为 2110，习为 739，[SEP]为 102
print('token_ids:',token_ids)
# 因为只有一个句子所以 segment_ids 都是 0
print('segment_ids:',segment_ids)

token_ids: [101, 3322, 1690, 2110, 739, 102]
segment_ids: [0, 0, 0, 0, 0, 0]


In [5]:
# 编码测试
token_ids, segment_ids = tokenizer.encode(sentence0,sentence1)
# 可以看到两个句子分词后的结果为：
# ['[CLS]', '机', '器', '学', '习', '[SEP]', '深', '度', '学', '习', [SEP]]
print('token_ids:',token_ids)
# 0 表示第一个句子的 token，1 表示第二个句子的 token
print('segment_ids:',segment_ids)

token_ids: [101, 3322, 1690, 2110, 739, 102, 3918, 2428, 2110, 739, 102]
segment_ids: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [6]:
# 增加一个维度表示批次大小为 1
token_ids = np.expand_dims(token_ids,axis=0)
# 增加一个维度表示批次大小为 1
segment_ids = np.expand_dims(segment_ids,axis=0)
# 传入模型进行预测
pre = model.predict([token_ids, segment_ids])
# 得到的结果中 1 表示批次大小，11 表示 11 个 token，768 表示特征向量长度
# 这里就是把句子的 token 转化为了特征向量
print(pre.shape)

1/1 [==============================] - 4s 4s/step
(1, 11, 768)


**完形填空**

In [7]:
# 建立模型，加载权重
# with_mlm=True 表示使用 mlm 的功能，模型结构及最后的输出会发生一些变化，可以用来预测被 mask 的 token
model = build_transformer_model(config_path, checkpoint_path, with_mlm=True)
# 分词并转化为编码
token_ids, segment_ids = tokenizer.encode('机器学习是一门交叉学科')
# 把“学”字和“习”字变成“[MASK]”符号
token_ids[3] = token_ids[4] = tokenizer._token_dict['[MASK]']
# 增加一个维度表示批次大小为 1
token_ids = np.expand_dims(token_ids,axis=0)
# 增加一个维度表示批次大小为 1
segment_ids = np.expand_dims(segment_ids,axis=0)
# 传入模型进行预测
pre = model.predict([token_ids, segment_ids])[0]
# 我们可以看到第 3，4 个位置经过模型预测，[MASK]变成了“学习”
print(tokenizer.decode(pre[3:5].argmax(axis=1)))

1/1 [==============================] - 2s 2s/step
学习


In [8]:
# 分词并转化为编码
token_ids, segment_ids = tokenizer.encode('机器学习是一门交叉学科')
# 把“交”字和“叉”字变成“[MASK]”符号
token_ids[8] = token_ids[9] = tokenizer._token_dict['[MASK]']
# 增加一个维度表示批次大小为 1
token_ids = np.expand_dims(token_ids,axis=0)
# 增加一个维度表示批次大小为 1
segment_ids = np.expand_dims(segment_ids,axis=0)
# 传入模型进行预测
pre = model.predict([token_ids, segment_ids])[0]
# 我们可以看到第 8，9 个位置经过模型预测，[MASK]变成了“什么”，句子变成了一个疑问句
# 虽然模型没有预测出原始句子的词汇，不过作为完形填空，填入一个“什么”句子也是正确
print(tokenizer.decode(pre[8:10].argmax(axis=1)))

1/1 [==============================] - 0s 143ms/step
什么
